In [1]:
import warnings

import torch
import torch.nn.functional as F

from torch import nn
from torch.utils.data import DataLoader, Dataset
from torchvision.transforms import Resize, Compose, ToTensor, Normalize

import argparse
import os
import math 
import skimage
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image

import time
import pickle

from datetime import datetime
from pathlib import Path

from py_files.new_dataset import *

from py_files.cnn_model import *
from py_files.pigan_model import *

from py_files.seq_pi_gan_functions import *

%matplotlib qt

c:\users\pault\appdata\local\programs\python\python37\lib\site-packages\kornia\augmentation\augmentation.py:1833: DeprecationWarning: GaussianBlur is no longer maintained and will be removed from the future versions. Please use RandomGaussianBlur instead.
  category=DeprecationWarning,


Imported CNN and Mapping functions.
Loaded all helper functions.


In [2]:
root = os.path.abspath('..')
root = os.path.join(root, "Dataset", "new_original")

for file in os.listdir(root):
    if "mask" in file:
        image = torch.tensor(np.load(os.path.join(root, file))).unsqueeze(0).unsqueeze(0).float()
        break
    

c:\users\pault\appdata\local\programs\python\python37\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [3]:
print(image.shape)

torch.Size([1, 1, 128, 128, 24])


c:\users\pault\appdata\local\programs\python\python37\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [16]:
maxpoollayer = torch.nn.MaxPool3d(3,stride=1,padding=1)

image_eroded = 1 - maxpoollayer(1 - image)

surface = image - image_eroded


c:\users\pault\appdata\local\programs\python\python37\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [17]:
sigma = 1
size = math.ceil(3*sigma)
blur_layer = GaussianSmoothing(1, [size,size,size], sigma, dim=3)

masks_blurred = blur_layer(image)     

grad, grad_magn = gradient3d(masks_blurred, normalize=True, s=2)

# surface = (grad_magn >= 0.5*torch.max(grad_magn)).type(torch.float32)
norm = grad * surface

c:\users\pault\appdata\local\programs\python\python37\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [18]:
window = Show_images("test", (image.squeeze(), "mask"), (surface.squeeze(), "surface"), 
                     (norm.squeeze()[0, :, :, :], "surface"), 
                     (norm.squeeze()[1, :, :, :], "surface"), 
                     (norm.squeeze()[2, :, :, :], "surface"),)

c:\users\pault\appdata\local\programs\python\python37\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
t = time.time()

print(surface.nonzero())

print(time.time() - t)
